In [1]:
import json # read from json file
import re
import sklearn
from sklearn.utils import shuffle
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
import eli5

#with open('data.jsonl') as jsonl_file: # not using the full (unannotated) dataset in the below
#    lines = jsonl_file.readlines()
#jsonS = [json.loads(line) for line in lines]
#print(len(jsonS))

In [2]:
### RUN
with open('annotations2.jsonl') as jsonl_file:
    # note: after running data-preprocessing.ipynb this file already has token-level labels
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
#print(annot[0])

Chose whether or not to add leading "B-" / "I-" to class labels

In [3]:
########## ONLY RUN IF WE WANT TO ADD LEADING "B-" / "I-" TO CLASS LABEL
# now use above code and loop through all items of annot list:
# addLeading=1 for "Yes" (i.e. add leading "B-","I-" to annot); 0 for "No" (i.e. add labels to annot simply as they are)
addLeading = 1

if addLeading == 1:
    for j in range(0,len(annot)):
        a = annot[j]
        # select list of dict of tokens w/ annnotations and add column w/ no. of words to each dict:
        b = a['spans']
        # add noWords to b dict. note: b is list of dicts w/ annotations; tokens not on this list don't have annotations
        if b!=[]: #i.e. only try to add annotations to tokens if there are annotations to begin with
            #print(b)
            for i in range(0,len(annot[j]['tokens'])):
                    # now break-up label into 1st occurrence (leading "B-") and subsequent occurrences (leading "I-") (only for non "O"'s)
                    if annot[j]['tokens'][i]['label'] != "O":
                        if i==0:
                            annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label']
                        else: 
                            if annot[j]['tokens'][i]['label'] == annot[j]['tokens'][i-1]['label'][2:]: # need to remove the leading "B-" that we had already been added to c[i-1]
                                annot[j]['tokens'][i]['label'] = "I-" + annot[j]['tokens'][i]['label']
                            else:
                                annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label'] 

In [4]:
### RUN
# now convert annotation tokens into list (sentences) of lists (tokens) format for sklearn_crfsuite.CRF
train_sents=[] 
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    train_sentence = []
    for i in range(0,len(a)):
        if 'label' in a[i]: # only add element if this sample sentence has been labelled 
            token_element = (a[i]['text'],a[i]['label'])
            train_sentence.append(token_element)
    train_sents.append(train_sentence)

In [5]:
print(train_sents[0])

[('DORNBIRN', 'B-ORT'), ('In', 'O'), ('der', 'O'), ('Schulgasse', 'B-STRASSE'), ('in', 'O'), ('Dornbirn', 'B-ORT'), ('hat', 'O'), ('eine', 'O'), ('71,93', 'B-FLAECHE'), ('Quadratmeter', 'O'), ('große', 'O'), ('Wohnung', 'B-IMMO_TYP'), ('für', 'O'), ('einen', 'O'), ('Quadratmeterpreis', 'O'), ('von', 'O'), ('5533,71', 'B-QMPREIS'), ('Euro', 'O'), ('den', 'O'), ('Besitzer', 'O'), ('gewechselt', 'O'), ('.', 'O'), ('Dieser', 'O'), ('beinhaltet', 'O'), ('auch', 'O'), ('einen', 'O'), ('Pkw-Abstellplatz', 'O'), ('.', 'O'), ('Käufer', 'O'), ('der', 'O'), ('Wohnung', 'O'), ('mit', 'O'), ('9,86', 'B-TERRASSENGROESSE'), ('Quadratmetern', 'O'), ('Terrasse', 'O'), ('ist', 'O'), ('die', 'O'), ('ValLiLean', 'B-KAEUFER'), ('Beteiligungs-', 'I-KAEUFER'), ('und', 'I-KAEUFER'), ('Immobilienverwaltungs', 'I-KAEUFER'), ('GmbH', 'I-KAEUFER'), ('.', 'O'), ('Beim', 'O'), ('Verkäufer', 'O'), ('handelt', 'O'), ('es', 'O'), ('sich', 'O'), ('um', 'O'), ('die', 'O'), ('Karrenblick', 'B-VERKAEUFER'), ('Projekt', 'I

In [6]:
print(annot[0]['spans'])

[{'text': 'DORNBIRN', 'start': 0, 'end': 8, 'pattern': 2069086582, 'token_start': 0, 'token_end': 0, 'label': 'ORT', 'noWords': 1}, {'start': 16, 'end': 26, 'token_start': 3, 'token_end': 3, 'label': 'STRASSE', 'noWords': 1}, {'text': 'Dornbirn', 'start': 30, 'end': 38, 'pattern': 2069086582, 'token_start': 5, 'token_end': 5, 'label': 'ORT', 'noWords': 1}, {'start': 48, 'end': 53, 'token_start': 8, 'token_end': 8, 'label': 'FLAECHE', 'noWords': 1}, {'start': 73, 'end': 80, 'token_start': 11, 'token_end': 11, 'label': 'IMMO_TYP', 'noWords': 1}, {'start': 113, 'end': 120, 'token_start': 16, 'token_end': 16, 'label': 'QMPREIS', 'noWords': 1}, {'start': 221, 'end': 225, 'token_start': 32, 'token_end': 32, 'label': 'TERRASSENGROESSE', 'noWords': 1}, {'start': 257, 'end': 311, 'token_start': 37, 'token_end': 41, 'label': 'KAEUFER', 'noWords': 5}, {'start': 351, 'end': 375, 'token_start': 49, 'token_end': 51, 'label': 'VERKAEUFER', 'noWords': 3}, {'text': '398.040', 'start': 402, 'end': 409, 

In [18]:
### RUN
import pandas as pd
df = pd.read_csv('townsVoralberg.csv',names=['towns'])  

In [19]:
### RUN
lCase = lambda a: str(a).lower()

def word2features(sent, i):
    word = sent[i][0]
    word_l = word.lower()
    towns = map(lCase,df['towns'])
    #postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word_l,
        'word[-3:]': word[-3:], # 3 char ending of word (suffix)
        'word[-2:]': word[-2:], # 2 char ending of word (suffix)
        'word.isupper()': word.isupper(), # all letters are caps
        'word.istitle()': word.istitle(), # 1st letter is a capital letter
        'word.isdigit()': word.isdigit(), # is digit?
        'word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word)),
        'word.isTown()': word_l in towns,
        #'postag': postag, # don't have PoS data
        #'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0] # enriches feature function w/ immediately preceding word
        word1_l = word1.lower()
        #postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1_l,
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word1)),
            '-1.word.isTown()': word1_l in towns,
            #'-1:postag': postag1,
            #'-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True # adds the start token

    if i < len(sent)-1:
        word1 = sent[i+1][0] # ... as well as w/ immediate sucessor word 
        word1_l = word1.lower()
        #postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1_l,
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word1)),
            '+1.word.isTown()': word1_l in towns,
            #'+1:postag': postag1,
            #'+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True # adds the end token

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
   # return [label for token, postag, label in sent]
    return [label for token, label in sent]

def sent2tokens(sent):
   # return [token for token, postag, label in sent]
    return [token for token, label in sent]

In [20]:
sent2features(train_sents[0])[2]

{'bias': 1.0,
 'word.lower()': 'der',
 'word[-3:]': 'der',
 'word[-2:]': 'er',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isPrice()': False,
 'word.isTown()': False,
 '-1:word.lower()': 'in',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:word.isPrice()': False,
 '-1.word.isTown()': False,
 '+1:word.lower()': 'schulgasse',
 '+1:word.istitle()': True,
 '+1:word.isupper()': False,
 '+1word.isPrice()': False,
 '+1.word.isTown()': False}

In [21]:
####### RUN THIS ONE
X = [sent2features(s) for s in train_sents] # both are 140 x # of tokens in each example
y = [sent2labels(s) for s in train_sents]

In [22]:
train_ratio = 0.75
train_test_split = round(0.75*len(X) - 0.5) # -0.5 => floor
idx = [i for i in range(0,len(X))]
idx_shuffle = shuffle(idx,random_state=0)
X_shuffle, y_shuffle = [X[auxIdx] for auxIdx in idx_shuffle], [y[auxIdx] for auxIdx in idx_shuffle]
X_train, X_test, y_train, y_test = X_shuffle[:train_test_split], X_shuffle[train_test_split:], y_shuffle[:train_test_split], y_shuffle[train_test_split:]

In [23]:
####### RUN THIS ONE
%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train[:120], y_train[:120])

Wall time: 0 ns


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [24]:
labels = list(crf.classes_)
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.9480150023989771

In [25]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                       precision    recall  f1-score   support

                    O      0.972     0.992     0.982      1472
B-DATUM_VERBUECHERUNG      0.935     0.935     0.935        31
I-DATUM_VERBUECHERUNG      0.931     1.000     0.964        27
      B-DATUM_VERTRAG      1.000     0.935     0.967        31
      I-DATUM_VERTRAG      1.000     0.909     0.952        33
            B-FLAECHE      1.000     0.974     0.987        38
            I-FLAECHE      0.000     0.000     0.000         0
        B-GESAMTPREIS      1.000     0.885     0.939        26
        I-GESAMTPREIS      0.000     0.000     0.000         3
           B-IMMO_TYP      0.947     0.837     0.889        43
           I-IMMO_TYP      0.400     0.571     0.471         7
            B-KAEUFER      0.522     0.571     0.545        21
            I-KAEUFER      0.318     0.583     0.412        12
                B-ORT      0.984     1.000     0.992        61
            B-QMPREIS      1.000     1.000     1.000  

C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['O', 'B-DATUM_VERBUECHERUNG', 'I-DATUM_VERBUECHERUNG', 'B-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'B-FLAECHE', 'I-FLAECHE', 'B-GESAMTPREIS', 'I-GESAMTPREIS', 'B-IMMO_TYP', 'I-IMMO_TYP', 'B-KAEUFER', 'I-KAEUFER', 'B-ORT', 'B-QMPREIS', 'I-QMPREIS', 'B-STRASSE', 'I-STRASSE', 'B-TERRASSENGROESSE', 'B-VERKAEUFER', 'I-VERKAEUFER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_cl

In [26]:
####### RUN THIS ONE
eli5.show_weights(crf, top=30)

From \ To,O,B-DATUM_VERBUECHERUNG,I-DATUM_VERBUECHERUNG,B-DATUM_VERTRAG,I-DATUM_VERTRAG,B-FLAECHE,I-FLAECHE,B-GESAMTPREIS,I-GESAMTPREIS,B-IMMO_TYP,I-IMMO_TYP,B-KAEUFER,I-KAEUFER,B-ORT,B-QMPREIS,I-QMPREIS,B-STRASSE,I-STRASSE,B-TERRASSENGROESSE,B-VERKAEUFER,I-VERKAEUFER
O,2.579,1.792,-1.644,1.707,-1.975,1.903,-0.865,1.561,-0.866,1.866,-3.169,1.485,-2.057,1.123,0.999,-0.7,1.536,-2.4,0.622,1.591,-2.229
B-DATUM_VERBUECHERUNG,-0.72,-0.477,4.247,0.0,-0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERBUECHERUNG,0.0,0.0,1.947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-DATUM_VERTRAG,-0.756,-0.04,-0.001,-0.448,4.767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERTRAG,0.414,0.0,-0.672,0.0,3.227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-FLAECHE,0.84,0.0,0.0,0.0,-0.228,0.0,1.527,0.0,0.0,-0.11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.128
I-FLAECHE,0.163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-GESAMTPREIS,0.857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-GESAMTPREIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-IMMO_TYP,0.343,0.0,-0.108,0.0,0.0,-0.114,0.0,0.0,0.0,-0.28,2.578,0.0,-0.147,0.0,-0.145,0.0,0.088,-0.066,0.0,0.0,-0.093


In [27]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-DATUM_VERTRAG -> I-DATUM_VERTRAG 4.766528
I-KAEUFER -> I-KAEUFER 4.443003
I-VERKAEUFER -> I-VERKAEUFER 4.384418
B-DATUM_VERBUECHERUNG -> I-DATUM_VERBUECHERUNG 4.246680
B-KAEUFER -> I-KAEUFER 4.175964
B-STRASSE -> I-STRASSE 4.091417
B-VERKAEUFER -> I-VERKAEUFER 4.082300
I-IMMO_TYP -> I-IMMO_TYP 3.754685
I-STRASSE -> I-STRASSE 3.651305
I-DATUM_VERTRAG -> I-DATUM_VERTRAG 3.226935
O      -> O       2.579295
B-IMMO_TYP -> I-IMMO_TYP 2.578381
I-DATUM_VERBUECHERUNG -> I-DATUM_VERBUECHERUNG 1.946697
O      -> B-FLAECHE 1.902822
O      -> B-IMMO_TYP 1.866466
O      -> B-DATUM_VERBUECHERUNG 1.792479
B-ORT  -> O       1.725265
O      -> B-DATUM_VERTRAG 1.706905
O      -> B-VERKAEUFER 1.591182
B-QMPREIS -> I-QMPREIS 1.581201

Top unlikely transitions:
B-DATUM_VERTRAG -> B-DATUM_VERTRAG -0.448054
B-DATUM_VERBUECHERUNG -> B-DATUM_VERBUECHERUNG -0.476693
I-STRASSE -> B-STRASSE -0.519746
B-KAEUFER -> I-VERKAEUFER -0.572626
I-DATUM_VERTRAG -> I-DATUM_VERBUECHERUNG -0.671965
O 

Below we collect various types of summary statistics of our labelled dataset